In [ ]:
import torch
import torchvision.models as models
import torchvision.transforms as transforms
from PIL import Image, UnidentifiedImageError
import os
import numpy as np
import pandas as pd

# 📌 Cargar modelo preentrenado (sin la última capa)
model = models.resnet18(pretrained=True)
model = torch.nn.Sequential(*list(model.children())[:-1])  # Quitar capa FC
model.eval()

# 📌 Transformaciones de imagen
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

# 📂 Directorios
data_dir = "Dataset"
features_dir = "features"
output_file = os.path.join(features_dir, "features.csv")

# 📌 Verificar si el directorio Dataset existe
if not os.path.exists(data_dir):
    print(f"⚠️ El directorio '{data_dir}' no existe.")
    exit()

# 📌 Crear carpeta 'features' si no existe
os.makedirs(features_dir, exist_ok=True)

# 🗑️ Verificar si el archivo features.csv ya existe
if os.path.exists(output_file):
    respuesta = input(f"📂 El archivo '{output_file}' ya existe. ¿Deseas sobrescribirlo? (s/n): ").strip().lower()
    if respuesta != "s":
        print("ℹ️ Operación cancelada.")
        exit()
    os.remove(output_file)
    print("✅ Archivo anterior eliminado.")

# 📌 Extraer características de cada imagen
features = []
labels = []

for img_name in os.listdir(data_dir):
    img_path = os.path.join(data_dir, img_name)

    try:
        image = Image.open(img_path).convert("RGB")
        image = transform(image).unsqueeze(0)  # Añadir batch dimension

        with torch.no_grad():
            embedding = model(image).flatten().numpy()  # Extraer feature vector

        features.append(embedding)
        labels.append(img_name.split('.')[0])  # Usar nombre de archivo como ID

    except UnidentifiedImageError:
        print(f"⚠️ Archivo ignorado (no es una imagen válida): {img_name}")

# 📌 Convertir a DataFrame y guardar
if features:
    df = pd.DataFrame(features)
    df["label"] = labels
    df.to_csv(output_file, index=False)
    print(f"✅ Features guardadas en '{output_file}'")
else:
    print("⚠️ No se encontraron imágenes válidas en el directorio.")




c:\Users\wotan\AppData\Local\Programs\Python\Python313\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\wotan\AppData\Local\Programs\Python\Python313\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


✅ Features guardadas en 'features.csv'
